# Building a dataset for NREL and LCD weather and solar data
This notebook is intended to be a first step in gathering data for a solar energy forecasting model. The final dataset will consist of hourly weather data and matching hourly solar data for multiple weather stations. This dataset can then be used for analytical analysis or to train a machine learning model.

## LCD data acquisition
We are going to first grab data from the [Local Climatological Data (LCD)](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/quality-controlled-local-climatological-data-qclcd) Dataset.
LCD dataset is a NCEI NOAA open database. It contains historic hourly weather data
for major weather stations in the US. Primarily what we are looking for is cloud coverage data to compare with the solar data later on. There are many other public and private datasets that could have been used instead of LCD, but this one is free and open to anyone. For more accurate data, a paid historical dataset could have been used.

To begin, first we need to set the parameter variables for selecting our data.

In [44]:
year = '2017' # 2017 is the max year
#, 'Iowa', 'Arizona', 'Tennessee', 'Oregon', 'Colorado', 'North Carolina', 'North Dakota'
states = ['Nevada'] # Use full capitalized names (any US state)
station_count = None # None = all stations

Now we can set up the url path and the station metadata that will be used after to extract the raw data using Pandas.

In [45]:
import pandas as pd

data_path = 'https://www.ncei.noaa.gov/data/local-climatological-data/access/'+ year +'/'
stations = pd.read_csv('stations.csv', dtype={"LATITUDE": float, "LONGITUDE": float})
stations = stations[stations['STATE'].isin(states)][:station_count]

print(stations.shape)
stations.head()

(22, 10)


,file,STATION_ID,STATION,BEGIN_DATE,END_DATE,STATE,COUNTRY,LATITUDE,LONGITUDE,ELEVATION_(M)
172,72054900171.csv,171,"CARSON AIRPORT, NV US",2010-07-13,2020-06-21,Nevada,United States,39.18300,-119.7330,1432.3
261,72074100269.csv,269,"BOULDER CITY MUNICIPAL AIRPORT, NV US",2010-06-23,2020-06-21,Nevada,United States,35.94700,-114.8610,671.2
272,72083900279.csv,279,"RENO STEAD AIRPORT, NV US",2011-09-20,2020-06-21,Nevada,United States,39.66700,-119.8760,1540.2
404,72209653127.csv,53127,"LAS VEGAS HENDERSON AIRPORT, NV US",2005-01-03,2020-06-21,Nevada,United States,35.97611,-115.1325,749.2
927,72386023169.csv,23169,"MCCARRAN INTERNATIONAL AIRPORT, NV US",1948-12-18,2020-06-21,Nevada,United States,36.07190,-115.1634,664.5


Before we get the data, we will create a lambda expression that will clean up floating point numbers by removing unwanted characters.

In [46]:
clean_float = lambda x: ''.join(filter(lambda y: y.isdigit() or y == '.', x))
clean_float('23.5s')

'23.5'

Next we will import tqdm. This library adds loading bar functionality
which will help visualize some of the larger Pandas operations.

In [47]:
from tqdm.auto import tqdm
tqdm.pandas()

/Users/gagekrumbach/.local/share/virtualenvs/Solar_Forcaster-m3btlaz4/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Finally we can access our data through the LCD database using HTTPS. We only extract a set amount of columns that we need. We also pass in our float cleaner for three of the columns.

In [48]:
from datetime import datetime as dt

w_data = pd.concat([pd.read_csv(data_path + station[1]['file'],
                            usecols=['STATION', 'DATE', 'HourlySkyConditions', 'HourlyStationPressure',
                                'HourlyVisibility', 'HourlyPresentWeatherType', 'HourlyPrecipitation',
                                    'HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
                                    'HourlyRelativeHumidity', 'HourlyWindSpeed'],
                            parse_dates=['DATE'],
                            dtype={'HourlySkyConditions': str, 'HourlyPresentWeatherType': str},
                            converters={'HourlyStationPressure': clean_float, 'HourlyVisibility': clean_float,
                                        'HourlyPrecipitation': clean_float, 'HourlyDewPointTemperature': clean_float,
                                       'HourlyDryBulbTemperature': clean_float, 'HourlyRelativeHumidity': clean_float,
                                       'HourlyWindSpeed': clean_float})
                         for station in tqdm(stations.iterrows(), total=stations.shape[0])\
                      if dt.strptime(station[1]["BEGIN_DATE"], '%Y-%m-%d') < dt.strptime(year, '%Y')\
                      and dt.strptime(station[1]["END_DATE"], '%Y-%m-%d') > dt.strptime(year, '%Y')])

 

## Data manipulation with Pandas

Now we will convert weather codes (ex: `-SN:03 BR:1 |SN |`) to a less specific form that can be used by the ML model later on. More information on these codes can be found [here](https://www.ncei.noaa.gov/data/local-climatological-data/doc/LCD_documentation.pdf). Codes are in the format `AU | AW | MW` with each observation separated by a bar. They need to be categorized into `freezing_rain_heavy, freezing_rainice_pellets, ice_pellets_light, tstorm` and other categories. We will perform this operation by using a json lookup file.

In [49]:
import json
with open('weather_lookup_converter.json') as json_file:
    lookup = json.load(json_file)

def weather_decoder(value):
    if pd.notnull(value):
        for weather_type in lookup:
            for code in lookup[weather_type]:
                if code in value:
                    return weather_type

# apply the decoder using a lambda expression
w_data['weather_type'] = w_data.progress_apply(lambda row: weather_decoder(row.HourlyPresentWeatherType), axis=1)

Next we do the same manipulation but with cloud observations (ex: `OVC:08 90`). We break up the hourly sky conditions column to a cloud string and cloud cover percentage using another lookup table. These will be categorized into `cloudy, partly_cloudy, clear` and other categories.

In [50]:
with open('cloud_lookup.json') as json_file:
    lookup = json.load(json_file)

def cloud_decoder(value):
    if pd.notnull(value):
        for cloud_type in lookup:
            if cloud_type in value:
                return lookup[cloud_type]
    return lookup["CLR"]
            
w_data['cloud_type'] = w_data.progress_apply(lambda row: cloud_decoder(row.HourlySkyConditions)["cloud_str"], axis=1)
w_data['cloud_cover'] = w_data.progress_apply(lambda row: cloud_decoder(row.HourlySkyConditions)["cloud_cover"], axis=1)

w_data.head()

,STATION,DATE,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPresentWeatherType,HourlyRelativeHumidity,HourlySkyConditions,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,weather_type,cloud_type,cloud_cover
0,72054900171,2017-01-01 00:15:00,23,28,,NaN,80,CLR:00,25.12,10.00,5,None,clear,0.0
1,72054900171,2017-01-01 00:35:00,23,28,,NaN,80,CLR:00,25.11,10.00,5,None,clear,0.0
2,72054900171,2017-01-01 00:55:00,23,28,,NaN,80,CLR:00,25.11,10.00,6,None,clear,0.0
3,72054900171,2017-01-01 01:15:00,21,27,,NaN,80,CLR:00,25.10,10.00,5,None,clear,0.0
4,72054900171,2017-01-01 01:35:00,21,27,,NaN,80,CLR:00,25.10,10.00,3,None,clear,0.0


## Combine meta data and actual data
Next we will combine the station metadata from before (lat, long, elevation) with the historic weather data. Before we do this, we need to alter the historic weather data station ids so they match the metadata station ids.<br/><br/>
The metadata uses a 5 digit WBAN id: `54852` and the historic data uses a 6 digit USAF MASTER id + the WBAN id: `72034354852`.

In [59]:
# remove USAF MASTER id
w_data['STATION'] = w_data['STATION'].map(lambda x: int(str(x)[-5:]))

In [ ]:
# add the meta data to the weather data
w_data["latitude"] = w_data.apply(lambda row:
                                      stations[stations["STATION_ID"] == row.STATION]['LATITUDE'], axis=1)

In [ ]:
w_data["longitude"] = w_data.apply(lambda row:
                                      float(stations[stations["STATION_ID"] == row.STATION]['LONGITUDE']), axis=1)

In [ ]:
w_data["elevation"] = w_data.apply(lambda row:
                                      float(stations[stations["STATION_ID"] == row.STATION]['ELEVATION_(M)']), axis=1)
w_data.head()

## Solar data acquisition
Now we will load the [NREL](https://nsrdb.nrel.gov/) data from the National Solar Radiation Database (NSREL). We use their api to get solar data for each station in our meta data. This will later be merged together to form a complete dataset.<br/><br/>
We will be using the Physical Solar Model ([PSM](https://nsrdb.nrel.gov/about/u-s-data.html#psm)) v3 that NSREL provides. To get an api key, follow the guide [here](https://developer.nrel.gov/docs/solar/nsrdb/psm3_data_download/), but the api key `DEMO_KEY` will also work fine. Keep in mind that this key only has a rate limit of 30 calls a day per IP address.

In [ ]:
# set api key (use 'DEMO_KEY' if not set up)
key = 'DEMO_KEY'

import os
key = os.getenv("MY_VAR")

# trim dataset to match DEMO_KEY limits
if(key == 'DEMO_KEY'):
    w_data = w_data[w_data["STATION"].isin(w_data['STATION'].unique()[:3])]
    print("Stations: " + str(w_data['STATION'].unique()))

Because the api only allows us to get one location per call, we will need to complete a series of calls. First we will create a function to create the urls.

In [ ]:
def create_url(lat, lon, year, api_key, attributes, your_name, reason_for_use, your_affiliation, your_email):
    return 'https://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day=false&interval=30&utc=false&full_name={name}&email={email}&affiliation={affiliation}&mailing_list=false&reason={reason}&api_key={api}&attributes={attributes}'\
    .format(year=year, lat=lat, lon=lon,
        name=your_name, email=your_email,
        affiliation=your_affiliation,
        reason=reason_for_use, api=api_key,
        attributes=attributes)

Now we will iterate through each station in our data by using Pandas' `.unique()` function. Then we will form the url and append the data into one list.

In [ ]:
import time
for step, point in enumerate(tqdm(stations[stations["STATION_ID"].isin(w_data['STATION'].unique())].iterrows(),
                                  total=len(w_data['STATION'].unique()))):
    url = create_url(lat=str(point[1][7]),
                     lon=str(point[1][8]),
                     api_key=key,
                     attributes="dhi,dni,ghi,clearsky_dhi,clearsky_dni,clearsky_ghi",
                     year=year,
                     your_name='Gage+Krumbach',
                     reason_for_use='demo',
                     your_affiliation='my+institution',
                     your_email='gkrumbac@redhat.com')
    
    output = pd.read_csv(url, header=2)
    
    output["STATION"] = int(point[1][1])
    output["DATE"] =pd.date_range('1/1/{yr}'.format(yr=year),
                                                    freq='30'+'Min',
                                                    periods=525600/60)
    if step==0:
        solar_data = output
    else:
        solar_data = solar_data.append(output)
    
    # we wait here because the api can't be called more than once in 1 seconds
    time.sleep(1)

solar_data.head()

## Merge solar data and weather data
Finally we can merge the solar data and weather data. We do this by using Pandas' `merge_asof()` function which will merge two dataframes by the nearest key. Meaning, a row with a date of `1-2-2017 11:34` will merge with a date `1-2-2017 11:55`, because it is the closest.

In [ ]:
df_merge_asof = pd.merge_asof(solar_data.sort_values(by=['DATE']), w_data.sort_values(by=['DATE']),
              on='DATE',
              by='STATION',
             direction='nearest')

## Altair visualization
We can use Altair to visualize the weather stations.

In [ ]:
import altair as alt
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, feature='states')

coord_center = [w_data['longitude'].unique().mean(),w_data['latitude'].unique().mean()]

background = alt.Chart(states).mark_geoshape(
    fill='#4D4D4D',
    stroke='white'
).project(
    type= 'mercator',
    scale= 500,
    center= coord_center
).properties(
    title='Stations',
    width=400, height=300
)

st = alt.Chart(stations[stations["STATION_ID"].isin(w_data['STATION'].unique())]
).mark_circle(
    size=45,
    color='white'
).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q',
    tooltip=['STATION:N']
).project(
    type= 'mercator',
    scale= 2000,
    center= coord_center
)

all_st = alt.Chart(pd.read_csv('stations.csv', dtype={"LATITUDE": float, "LONGITUDE": float})
).mark_circle(
    size=10,
    color='#FAB6B6'
).encode(
    longitude='LONGITUDE:Q',
    latitude='LATITUDE:Q'
    ).project(
    type= 'mercator',
    scale= 2000,
    center= coord_center
)

background + all_st + st

To cleanup the data further, we only select the columns that are relevant.

In [ ]:
df_merge_asof = df_merge_asof[['STATION', 'DATE', 'latitude', 'longitude', 'elevation',
                               'Temperature', 'Dew Point', 'Relative Humidity', 'HourlyStationPressure',
                               'Wind Direction', 'Wind Speed', 'HourlyVisibility', 'HourlyPrecipitation',
                               'weather_type', 'cloud_type', 'cloud_cover', 'DHI', 'DNI', 'GHI',
                               "Clearsky DHI","Clearsky DNI","Clearsky GHI"
                              ]]
df_merge_asof.head()

We take this final data and export it back out as a csv.

In [ ]:
df_merge_asof.to_csv("out.csv",index=False)